<a href="https://colab.research.google.com/github/ObaiMohammad/CV/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

import scipy.io
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [12]:
def prepare_eis_data(Zmatrix, soc_values, temperature_values,scale_real,scale_img):
    data_list_in = []
    data_list_out = []

    for i, soc in enumerate(soc_values):
        real_part = np.real(Zmatrix[:, :, i]) /scale_real
        imag_part = np.imag(Zmatrix[:, :, i]) /scale_img

        for col_idx, temp in enumerate(temperature_values):
            real_values = real_part[:, col_idx]
            imag_values = imag_part[:, col_idx]

            combined_features = list(real_values) + list(imag_values)
            combined_features.append(soc)

            data_list_in.append(combined_features)
            data_list_out.append([temp])

    df_in = pd.DataFrame(data_list_in)
    df_out = pd.DataFrame(data_list_out, columns=['Temperature'])

    return df_in, df_out

In [19]:
mat_data = scipy.io.loadmat('/content/drive/MyDrive/Data/EISresults.mat')
Zmatrix = mat_data['Zmatrix']

soc_values = [35, 55, 80]
temperature_values = [17, 28.1, 32.2, 48.1, 58]

df_in, df_out = prepare_eis_data(Zmatrix, soc_values, temperature_values,1000, -1000)

In [21]:
input = df_in
output = df_out

print("input.shape ",input.shape)
print("output.shape ",output.shape)

scaler_input = MinMaxScaler()
scaler_output = MinMaxScaler()

normalized_input  = pd.DataFrame(scaler_input.fit_transform(input))
normalized_out = pd.DataFrame(scaler_output.fit_transform(output),columns=['Temperature'])

input.shape  (15, 201)
output.shape  (15, 1)


In [22]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from scipy.stats import uniform

import joblib

In [24]:
# Load the saved model
# model_path = '/content/drive/MyDrive/Data/best_mlp_model_WN_SPLIt.pkl'
model_path_wn = '/content/drive/MyDrive/Data/best_mlp_model_WN_SPLIT_NEW_OUT.pkl'
model_path = '/content/drive/MyDrive/Data/best_mlp_model_No_WN_SPLIT_NEW_OUT.pkl'


best_mlp= joblib.load(model_path)

print("Model loaded successfully!")

Model loaded successfully!


In [25]:
best_mlp_wn= joblib.load(model_path_wn)

In [ ]:
best_mlp.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (200,),
 'learning_rate': 'constant',
 'learning_rate_init': np.float64(0.010029122914019804),
 'max_fun': 15000,
 'max_iter': 3000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [26]:
from sklearn.model_selection import cross_val_score

# Evaluate final model with cross-validation (optional)
cv_scores = cross_val_score(best_mlp, normalized_input, normalized_out, cv=5, scoring='neg_mean_squared_error')
print(f"CV MSE scores: {-cv_scores}")
print(f"Mean CV MSE: {-cv_scores.mean()}")


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed w

CV MSE scores: [0.00543208 0.0460082  0.02834855 0.01300539 0.11492171]
Mean CV MSE: 0.041543185254219864


In [27]:
from sklearn.metrics import mean_squared_error, r2_score


# Get Predictions
predictions = best_mlp.predict(normalized_input)

# Calculate R² Score for Each Target
r2_temp = r2_score(normalized_out, predictions)
print(f"R² Score for Temperature: {r2_temp}")

# Inverse Transform Predictions
predictions_original_scale = scaler_output.inverse_transform(predictions.reshape(-1, 1)).ravel()
true_values_original_scale = scaler_output.inverse_transform(normalized_out).ravel()
diff = true_values_original_scale-predictions_original_scale

# Compare Predictions with True Values
comparison_df = pd.DataFrame({
    'True Temperature': true_values_original_scale,
    'Predicted Temperature': predictions_original_scale,
    'diff':diff
})
print()
print(comparison_df)


R² Score for Temperature: 0.9378134414672437

    True Temperature  Predicted Temperature      diff
0               17.0              15.029487  1.970513
1               28.1              25.142443  2.957557
2               32.2              31.615096  0.584904
3               48.1              44.277957  3.822043
4               58.0              50.864231  7.135769
5               17.0              18.816322 -1.816322
6               28.1              29.069466 -0.969466
7               32.2              39.666726 -7.466726
8               48.1              51.096129 -2.996129
9               58.0              57.073291  0.926709
10              17.0              19.109455 -2.109455
11              28.1              28.506617 -0.406617
12              32.2              38.179144 -5.979144
13              48.1              48.830717 -0.730717
14              58.0              54.895516  3.104484
